Although there are some Python packages (wikipediaapi, wikipedia) that conveniently access Wikipedia, they are not powerful enough for our purposes.

In [38]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import sys
sys.path.append('../src/')
import wiki_api
import data_layer
import graph_api
from importlib import reload
#raph_api = reload(graph_api)

In [2]:
_, wiki_concepts = data_layer.read_index_and_wiki_concepts()

df_concepts = pd.DataFrame({"concept": list(wiki_concepts)})

In [3]:
df_concepts['links'] = [wiki_api.get_cached_links(concept) for concept in tqdm(df_concepts.concept)]

  6%|▌         | 85/1464 [00:27<07:35,  3.03it/s]

person detected on https://en.wikipedia.org/wiki/Hypatia


  7%|▋         | 106/1464 [00:33<05:07,  4.42it/s]

person detected on https://en.wikipedia.org/wiki/Baden_Powell_(mathematician)


  8%|▊         | 124/1464 [00:39<05:12,  4.29it/s]

...

 91%|█████████▏| 1337/1464 [07:22<00:23,  5.41it/s]

person detected on https://en.wikipedia.org/wiki/Andrey_Markov


 92%|█████████▏| 1340/1464 [07:22<00:21,  5.88it/s]

 unexpected page content on https://en.wikipedia.org/wiki/Congruent_transformation


100%|██████████| 1464/1464 [08:02<00:00,  3.03it/s]


In [4]:
count = Counter(sum([[x[1] for x in v] for v in df_concepts.links if v is not None], []))
count.most_common(30)

[('Mathematics', 639),
 ('Real number', 140),
 ('Vector space', 133),
 ('Function (mathematics)', 129),
 ('Matrix (mathematics)', 116),
 ('Linear algebra', 115),
 ('Geometry', 101),
 ('Set (mathematics)', 100),
 ('Field (mathematics)', 95),
 ('Physics', 94),
 ('Complex number', 91),
 ('Integer', 76),
 ('Polynomial', 72),
 ('Euclidean space', 67),
 ('Statistics', 57),
 ('Ring (mathematics)', 56),
 ('Computer science', 52),
 ('Group (mathematics)', 45),
 ('Engineering', 43),
 ('Algebra', 42),
 ('Algorithm', 41),
 ('Number theory', 39),
 ('Variable (mathematics)', 39),
 ('Quantum mechanics', 39),
 ('Square matrix', 38),
 ('Mathematical analysis', 38),
 ('Abstract algebra', 38),
 ('Euclidean geometry', 37),
 ('Numerical analysis', 37),
 ('Algebraic geometry', 36)]

In [5]:
blacklist = ['Mathematics', 'Linear algebra', 'Physics', 'Mathematical object', 'Engineering','Geometry','Statistics','Computer science', 'Algebra', 'Number theory','Algebraic geometry','Euclidean geometry', None]

In [18]:
def has_earlier_backlink(concept, dep_title, before_pos):
    links = wiki_api.get_cached_links(dep_title)
    backlink = list(filter(lambda bl: bl[1] == concept, links))
    return backlink[0][0] > before_pos if len(backlink) else False

# Now we prune the candidates
def prune_deps(concept, deps, titles, k=5):
    # 1. Blacklist
    deps = list(filter(lambda dep: dep[1] not in blacklist, deps))
    # 2. Link to another concept in our set of concepts
    #deps = filter(lambda dep: dep[1] in titles, deps)
    # 3. Position in summary
    first_paragraph_with_link = np.min([dep[2] for dep in deps]) if len(deps) else -1
    first_sentence_with_link = np.min([dep[3] for dep in deps if dep[2] == first_paragraph_with_link]) if first_paragraph_with_link >= 0 else -1
    deps = list(filter(lambda dep: dep[2] == first_paragraph_with_link and dep[3]==first_sentence_with_link, deps))
    # 4. dependencies on unexpected pages and persons
    deps = list(filter(lambda dep: wiki_api.get_cached_links(dep[1]) is not None, deps))
    # 5. Earlier Backlink: check if these articles link earlier to this article to delete cycles
    deps = list(filter(lambda dep: not has_earlier_backlink(concept, dep[1], dep[0]), deps))
    # 5. maximum number of dependencies
    deps = deps[:min(k, len(deps))]
    print(concept + ": "+ ", ".join([dep[1] for dep in deps]))
    return deps
    
df_concepts = df_concepts.dropna()
df_concepts['dep_articles'] = df_concepts.apply(lambda c: prune_deps(c.concept, c.links, df_concepts.concept), axis=1)

Intersection: Euclidean geometry, Line (geometry), Plane (geometry), Point (geometry)
Singularity (mathematics): Well-behaved, Derivative, Analyticity of holomorphic functions
Discriminant: Polynomial, Coefficient, Zero of a function
Fractal: Topological dimension
Pseudovector: Function (mathematics), Geometric shape, Additive inverse, Orientation (space), Rigid transformation
Vector calculus: Derivative, Integral, Vector field, Euclidean space, Partial derivative
Transpose of a linear map: Linear map, Field (mathematics), Dual space
Crank–Nicolson method: Numerical analysis, Finite difference method, Heat equation, Partial differential equations, Big O notation
Jordan normal form: Upper triangular matrix, Jordan matrix, Linear operator, Finite-dimensional, Vector space
Google: Multinational corporation, Technology company, Search Engine, Online advertising, Cloud computing
Line (geometry): Curvature, One-dimensional space, Three-dimensional space, Line segment, Point (geometry)
Confor

In [39]:
df_concepts.to_csv("../dat/wiki/graph.csv")

graph_api.save_graph('../dat/wiki/graph.json', df_concepts.concept.values, df_concepts.dep_articles.values)